# 5 Data Segregation

In this section we are to split our dataset into two blocks. The first one is going to be used for traning and stored as training_data.csv. The other will be reserved for testing the model.

## 5.1 Login to WandB

Run:

    wandb login --relogin

## 5.2 Loading libraries

In [ ]:
import logging
import tempfile
import pandas as pd
import os
import wandb
from sklearn.model_selection import train_test_split

## 5.3 Data Segregation

In [ ]:
# global variables

# ratio used to split train and test data
test_size = 0.20

# seed used to reproduce purposes
seed = 57

# reference column to straitify the data
stratify = "status"

# name of the input artifact
artifact_input_name = "phishing-detection/preprocessed_data.csv:latest"

# type of the artifact
artifact_type = "data"

In [ ]:
# configure logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%Y-%m-%d %H:%M:%S')

# reference for a logging obj
logger = logging.getLogger()

# initiate wandb project
run = wandb.init(project="phishing-detection", job_type="split_data")

logger.info("Downloading and reading artifact")
artifact = run.use_artifact(artifact_input_name)
artifact_path = artifact.file()
df = pd.read_csv(artifact_path)

# Split firstily in train/test, then we further divide the dataset to train and validation
logger.info("Splitting data into train and test")
splits = {}
splits["train"], splits["test"] = train_test_split(df,
                                                   test_size=test_size,
                                                   random_state=seed,
                                                   stratify=df[stratify])

# Save the artifacts. We use a temporary directory so we do not leave any trace behind
with tempfile.TemporaryDirectory() as tmp_dir:
    
    for split, df in splits.items():

        # MAke the artifact name from the name of the split plus the provided root
        artifact_name = f"{split}.csv"

        # Get the path on disk within the tempo directory
        temp_path = os.path.join(tmp_dir, artifact_name)

        logger.info(f"Uploading the {split} dataset to {artifact_name}")

        # Save then upload to W&B
        df.to_csv(temp_path,index=False)

        artifact = wandb.Artifact(name=artifact_name,
                                  type=artifact_type,
                                  description=f"{split} split of dataset {artifact_input_name}")
        artifact.add_file(temp_path)

        logger.info("Logging artifact")
        run.log_artifact(artifact)

        # This waits for the artifact to be upload to W&B. If you do not add
        # this, te temp directory might be removed before W&B had a chance to 
        # upload the datasets, and upload might fail
        artifact.wait()

In [ ]:
# close the run
run.finish()